# Miscellaneous investigations into the stochastic Lie algebra

## Poormans symbolic Pade approximation

Using the representation of rationals as pairs of arbitrary precision integers we can effectively symbolic compute the Pade approximation of Taylor series with rational coefficients.

We will use the Polynomials packages, along with the base rational numbers, and arbitrary precision integers.

In [34]:
using Polynomials
using Gadfly

In this study we will formulate the Taylor series with coefficients given by inverse of the factorial of the arithmetic progression, with $a > 0$ and $b \ge 0$:
$$ 
\sum_{n=0}^\infty \frac{x^n}{\left(a\cdot n+b\right)!}
$$
In particular we are interested in $a=1$ and $b=2$

To compute the Pade coefficients we will use the following procedure:

1. Cast the series as rational arbitrary precision numbers
2. Then construct a polynomial
3. Finally find the rational Pade approximation

For ease of use we can turn this into a function to compute the *coefficients of the Pade approximation of the inverse arithmetic progression*:

In [35]:
padeprogression(n,m,a,b)=Pade(Poly([Rational(1,factorial(BigInt(a*k+b))) for k = 0:(n+m+1)]), n, m)

padeprogression (generic function with 1 method)

We can test the function

In [37]:
b=padeprogression(8,8,1,2)

Polynomials.Pade{Rational{BigInt},Rational{BigInt}}(Poly(1//2 - 1//18x + 49//3672x^2 - 1//1530x^3 + 1//14688x^4 - 1//771120x^5 + 1//13366080x^6 + 1//158789030400x^8),Poly(1//1 - 4//9x + 14//153x^2 - 7//612x^3 + 7//7344x^4 - 1//18360x^5 + 1//477360x^6 - 1//20049120x^7 + 1//1764322560x^8),:x)

Exploring the numerator

In [38]:
b.p.a

9-element Array{Rational{BigInt},1}:
  1//2           
 -1//18          
 49//3672        
 -1//1530        
  1//14688       
 -1//771120      
  1//13366080    
  0//1           
  1//158789030400

Exploring the denominator

In [39]:
b.q.a

9-element Array{Rational{BigInt},1}:
  1//1         
 -4//9         
 14//153       
 -7//612       
  7//7344      
 -1//18360     
  1//477360    
 -1//20049120  
  1//1764322560

For comparison here is the largest float, we have a little ways to go

In [40]:
[collect(62:64) [2^n - 1 for n = 62:64] [2^n for n = 62:64]]

3x3 Array{Int64,2}:
 62  4611686018427387903   4611686018427387904
 63  9223372036854775807  -9223372036854775808
 64                   -1                     0

In [42]:
c=padeprogression(12,13,1,2)

Polynomials.Pade{Rational{BigInt},Rational{BigInt}}(Poly(1//2 - 2//27x + 11//648x^2 - 11//8100x^3 + 1//7200x^4 - 1//144900x^5 + 1//2484000x^6 - 1//78246000x^7 + 1//2253484800x^8 - 1//117744580800x^9 + 1//6279710976000x^10 - 1//693908062848000x^11 + 1//116576554558464000x^12),Poly(1//1 - 13//27x + 1//9x^2 - 11//675x^3 + 11//6480x^4 - 11//82800x^5 + 1//124200x^6 - 1//2608200x^7 + 1//69552000x^8 - 1//2378678400x^9 + 1//107040528000x^10 - 1//6672192912000x^11 + 1//640530519552000x^12 - 1//124903451312640000x^13),:x)

In [43]:
[c.p.a[end].den c.q.a[end].den; 2^63 - 1 2^63 - 1 ; c.p.a[end].den - (2^63 - 1) c.q.a[end].den - (2^63 - 1)]

3x2 Array{BigInt,2}:
   116576554558464000    124903451312640000
  9223372036854775807   9223372036854775807
 -9106795482296311807  -9098468585542135807

## Logarithms of permutation matrices

Every permutation matrix is the product of cyclic permutation matrices; so we start be writing a function to construct the $k$ of $\left(n-1\right)!$ cyclic permutations of length $n$. Starting with the dense call:

In [1]:
function cyclicpermutation{T<:Number, S<:Integer}(::Type{T}, n::S, k::S)
    if n < 2 || k < 1
        throw(ArgumentError("Invalid dimension size or permutation ordinal."))
    end
    u = [1; nthperm(collect(2:n), BigInt(k))]
    A = zeros(T, n, n)
    a = convert(T, 1)
    for i = 1:n
        @inbounds A[u[i], u[mod(i, n) + 1]] = a
    end
    return A
end
cyclicpermutation{S<:Integer}(n::S, k::S) = cyclicpermutation(Int64, n, k)

cyclicpermutation (generic function with 2 methods)

...and the sparse version:

In [5]:
function spcyclicpermutation{T<:Number, S<:Integer}(::Type{T}, n::S, k::S)
    if n < 2 || k < 1
        throw(ArgumentError("Invalid dimension size or permutation ordinal."))
    end
    u = [1; nthperm(collect(2:n), BigInt(k))]
    v = [u[2:end]; 1]
    w = ones(T, n)
    sparse(u, v, w, n, n)
end
spcyclicpermutation{S<:Integer}(n::S, k::S) = spcyclicpermutation(Int64, n, k)

spcyclicpermutation (generic function with 2 methods)

A couple of quick tests generating permutation matrices $A_{\pi\left(\cdot\right)}$

In [4]:
A = cyclicpermutation(5,1)

5x5 Array{Int64,2}:
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1
 1  0  0  0  0

In [6]:
det(A)

1.0

...and the sparse call

In [12]:
B = spcyclicpermutation(6, 120)

6x6 sparse matrix with 6 Int64 entries:
	[2, 1]  =  1
	[3, 2]  =  1
	[4, 3]  =  1
	[5, 4]  =  1
	[6, 5]  =  1
	[1, 6]  =  1

In [10]:
det(B)

-1.0

The permutation matrices $A_{\pi\left(\cdot\right)}$ are related to the canonical generators of the stochastic Lie algebra through $A_{\pi\left(\cdot\right)} = C_{\pi\left(\cdot\right)} + I$, where:
$$
    \begin{eqnarray}
        C_{\pi\left(\cdot\right)}
            & = & \sum_{i=1}^n C_{i \pi\left(i\right)}\\
            & = & \sum_{i=1}^n \hat{e}_i \otimes \hat{e}_{\pi\left(i\right)} - \hat{e}_i \otimes \hat{e}_i
    \end{eqnarray}
$$

For example:

In [44]:
full(B) - I

6x6 Array{Int64,2}:
 -1   0   0   0   0   1
  1  -1   0   0   0   0
  0   1  -1   0   0   0
  0   0   1  -1   0   0
  0   0   0   1  -1   0
  0   0   0   0   1  -1

Lets take a look at some logarithms:

In [45]:
C = cyclicpermutation(4, 1)

4x4 Array{Int64,2}:
 0  1  0  0
 0  0  1  0
 0  0  0  1
 1  0  0  0

In [46]:
round(logm(C), 5)

4x4 Array{Complex{Float64},2}:
     0.0+0.7854im   0.7854-0.7854im     -0.0+0.7854im  -0.7854-0.7854im
 -0.7854-0.7854im      0.0+0.7854im   0.7854-0.7854im      0.0+0.7854im
     0.0+0.7854im  -0.7854-0.7854im      0.0+0.7854im   0.7854-0.7854im
  0.7854-0.7854im     -0.0+0.7854im  -0.7854-0.7854im      0.0+0.7854im

By definition the $\ln A_{\pi\left(\cdot\right)}$ must belong to the same Lie subalgebra as $C_{\pi\left(\cdot\right)} = A_{\pi\left(\cdot\right)} - I$.

Furthermore the Lie subalgebra of $\ln A_{\pi\left(\cdot\right)}$ must contain the positive powers $C_{\pi\left(\cdot\right)}^n$, which are given by the binomial sums:
$$
    C_{\pi\left(\cdot\right)}^n = \sum_{k=0}^n \left(-1\right)^{n-k} \binom{n}{k} C_{\pi^n \left(\cdot\right)}
$$
Where $C_{\pi^0 \left(\cdot\right)} = 0$.

As well we trivially have that $\left[C_{\pi^n \left(\cdot\right)}, C_{\pi^m \left(\cdot\right)}\right] = 0$ because $A_{\pi\left(\cdot\right)}^n=A_{\pi^n \left(\cdot\right)}$

Finally if $\pi \left(\cdot\right)$ has period $p$, so that $\pi^p \left(i\right) = i$ then
$$
    C_{\pi^n \left(\cdot\right)} = C_{\pi^{n\bmod p} \left(\cdot\right)}
$$

From which we can deduce that:
$$
    A_{\pi\left(\cdot\right)} = \prod_{n=1}^{p-1} \exp\left(\alpha_n C_{\pi^n \left(\cdot\right)} \right)
$$

Critically the coefficients $\alpha_n$ are independent of $\pi\left(\cdot\right)$, and depend only on $p$.

We start with a function to calculate coefficients based on the Pythagorean approximations of $\pi$. For even periods $p$ we return the coefficients rooted on the outer circle approximation, and for odd we return the coefficients rooted on the inner circle apporximations

In [13]:
function pythagoreancoefficients{S<:Integer, T<:Real}(::Type{T}, n::S)
    u = Vector{Complex{T}}((n - 1))
    nT = convert(T, n)
    piT = convert(T, pi)
    oT = convert(T, 1)
    noT = convert(T, -1)
    zT = convert(T, 0)
    sT = noT
    aT = zT
    if n % 2 == 0
        for m = 1:(n - 1)
            sT *= noT
            aT += oT
            @inbounds u[m] = complex(sT * piT / (nT * tan(piT * (aT / nT))), sT * piT / nT)
        end
    else
        for m = 1:(n - 1)
            sT *= noT
            aT += oT
            @inbounds u[m] = complex(sT * piT / (nT * sin(piT * (aT / nT))), zT)
        end
    end
    return u
end
pythagoreancoefficients{S<:Integer}(n::S) = pythagoreancoefficients(Float64, n::S)

pythagoreancoefficients (generic function with 2 methods)

Another quick test of the new functions

In [19]:
pythagoreancoefficients(11)

10-element Array{Complex{Float64},1}:
   1.01372+0.0im
 -0.528261+0.0im
  0.377902+0.0im
 -0.313972+0.0im
  0.288536+0.0im
 -0.288536+0.0im
  0.313972+0.0im
 -0.377902+0.0im
  0.528261+0.0im
  -1.01372+0.0im

We can implement this in a function that will always return a dense complex matrix:

In [27]:
function logcyclicpermutation{S<:Integer, T<:Real}(::Type{T}, n::S, k::S)
    A = cyclicpermutation(Complex{T}, n, k)
    B = zeros(A)
    C = eye(A)
    u = pythagoreancoefficients(T, n)
    for m = 1:(n - 1)
        C *= A
        @inbounds B += (C - I) * u[m]
    end
    return B
end
logcyclicpermutation{S<:Integer}(n::S, k::S) = logcyclicpermutation(Float64, n, k)

logcyclicpermutation (generic function with 2 methods)

Testing the function:

In [30]:
logcyclicpermutation(8,1)[1,:]'

8x1 Array{Complex{Float64},2}:
 -1.11022e-16+0.392699im
     0.948059-0.392699im
    -0.392699+0.392699im
     0.162661-0.392699im
 -2.40459e-17+0.392699im
    -0.162661-0.392699im
     0.392699+0.392699im
    -0.948059-0.392699im

In [31]:
logm(cyclicpermutation(8,1))[1,:]'

8x1 Array{Complex{Float64},2}:
 -3.33067e-16+0.392699im
     0.948059-0.392699im
    -0.392699+0.392699im
     0.162661-0.392699im
 -1.77636e-15+0.392699im
    -0.162661-0.392699im
     0.392699+0.392699im
    -0.948059-0.392699im